<a href="https://colab.research.google.com/github/kovzanok/dls-final-task/blob/main/1_dop_Identification_Rate_Metric.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Identification Rate Metric

При обучении модели для распознавания лиц с помощью CE (кросс-энтропии) мы можем считать метрику accuracy как индикатор того, насколько хорошо наша модель работает. Но у accuracy тут есть недостаток: она не сможет померить, насколько хорошо наша модель работает на лицах людей, которых нет в обучающей выборке.  

Чтобы это исправить, придумали новую метрику: **identification rate**. Вот как она работает:

Создадим два набора изображений лиц: query и distractors. Никакие лица из этих наборов не должны содержаться в обучающем и валидационном датасете.

1. посчитаем косинусные расстояния между лицами, соответствующими одним и тем же людям из query части. Например, пусть одному человеку соответствуют три фото в query: 01.jpg, 02.jpg, 03.jpg. Тогда считаем три косинусных расстояния между всеми тремя парами из этих фото.
2. посчитаем косинусные расстояния между лицами, соответствующими разным людям из query части.
3. посчитаем косинусные расстояния между всеми парами лиц из query и distractors. Т.е. пара — это (лицо из query, лицо из distractors). Всего получится |query|*|distractors| пар.
4. Сложим количества пар, полученных на 2 и 3 шагах. Это количество false пар.
5. Зафиксируем **FPR** (false positive rate). Пусть, например, будет 0.01. FPR, умноженный на количество false пар из шага 4 — это разрешенное количество false positives, которые мы разрешаем нашей модели. Обозначим это количество через N.
6. Отсортируем все значения косинусных расстояний false пар. N — ое по счету значение расстояния зафиксируем как **пороговое расстояние**.
7. Посчитаем количество positive пар с шага 1, которые имеют косинусное расстояние меньше, чем пороговое расстояние. Поделим это количество на общее количество positive пар с шага 1. Это будет TPR (true positive rate) — итоговое значение нашей метрики.

Такая метрика обычно обозначается как TPR@FPR=0.01. FPR может быть разным. Приразных FPR будет получаться разное TPR.

Смысл этой метрики в том, что мы фиксируем вероятность ошибки вида false positive, т.е. когда "сеть сказала, что это один и тот же человек, но это не так", считаем порог косинусного расстояния для этого значения ошибки, потом берем все positive пары и смотрим, у скольких из них расстояние меньше этого порога. Т.е. насколько точно наша сеть ищет похожие лица при заданной вероятности ошибки вида false positive.

**Для подсчета метрик, то вам нужно разбить данные на query и distractors самим.**

Делается это примерно так:
- Выбраете несколько id, которые не использовались при тренировке моделей, и помещаете их в query set;
- Выбираете несколько id, которые не использовались при тренировке моделей и не входят в query, и помещаете их в distractors set. Обычно distractors set должен быть сильно больше, чем query set.
- Обрабатываете картинки из query и distractors тем же способом, что картинки для обучения сети.


Обратите внимание, что если картинок в query и distractors очень много, то полученных пар картинок в пунктах 1-2-3 алгоритма подсчета TPR@FPR будет очень-очень много. Чтобы код подсчета работал быстрее, ограничивайте размеры этих датасетов. Контролируйте, сколько значений расстояний вы считаете.

Ниже дан шаблон кода для реализации FPR@TPR метрики и ячейки с тестами. Тесты проверяют, что ваш код в ячейках написан правильно.

## План заданий

* Правильно разбить датасет на query и distractors
* Реализовать метрику и пройти все тесты
* Подгрузить все модели, обученные на разных лоссах и сравнить их метрики

#Импорт зависимостей

In [31]:
import os
import cv2
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torch.nn import functional as F
from torch import nn
import torchvision.models as models
import torch
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import torchvision.utils as vutils
import torchvision.transforms.functional as TF

from itertools import combinations, chain
import math

In [24]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

#Скачиваем датасеты и все необходимый файлы

Полный датасет CelebA

In [3]:
!gdown 1HT72tZVCrXD0u_Ata8T1BaIMrz0rOcZn

Downloading...
From (original): https://drive.google.com/uc?id=1HT72tZVCrXD0u_Ata8T1BaIMrz0rOcZn
From (redirected): https://drive.google.com/uc?id=1HT72tZVCrXD0u_Ata8T1BaIMrz0rOcZn&confirm=t&uuid=2f29d15d-22d4-4a7a-aff6-90d0bc203f79
To: /content/img_align_celeba.zip
100% 1.44G/1.44G [00:16<00:00, 89.5MB/s]


In [4]:
!unzip -q /content/img_align_celeba.zip -d /content/celeba/

Датасет, использованный для обучения модели

In [5]:
!gdown 1bHLaSZ2frNjyK2hLTXTuPFlFRxfJDxwb

Downloading...
From (original): https://drive.google.com/uc?id=1bHLaSZ2frNjyK2hLTXTuPFlFRxfJDxwb
From (redirected): https://drive.google.com/uc?id=1bHLaSZ2frNjyK2hLTXTuPFlFRxfJDxwb&confirm=t&uuid=7649ade0-3fc5-4ce5-8b1c-e1ec36045f17
To: /content/file.zip
100% 401M/401M [00:05<00:00, 71.0MB/s]


In [6]:
!unzip /content/file.zip -d /content/

Streaming output truncated to the last 5000 lines.
  inflating: /content/content/stage_3_dataset/112660.jpg  
  inflating: /content/content/stage_3_dataset/129052.jpg  
  inflating: /content/content/stage_3_dataset/151389.jpg  
  inflating: /content/content/stage_3_dataset/118111.jpg  
  inflating: /content/content/stage_3_dataset/000174.jpg  
  inflating: /content/content/stage_3_dataset/013829.jpg  
  inflating: /content/content/stage_3_dataset/107408.jpg  
  inflating: /content/content/stage_3_dataset/112157.jpg  
  inflating: /content/content/stage_3_dataset/119239.jpg  
  inflating: /content/content/stage_3_dataset/120061.jpg  
  inflating: /content/content/stage_3_dataset/147590.jpg  
  inflating: /content/content/stage_3_dataset/037708.jpg  
  inflating: /content/content/stage_3_dataset/072505.jpg  
  inflating: /content/content/stage_3_dataset/100668.jpg  
  inflating: /content/content/stage_3_dataset/118256.jpg  
  inflating: /content/content/stage_3_dataset/121771.jpg  
  inf

Файл с парами "имя картинки" - id и создаем DataFrame

In [7]:
!gdown 1pmjLR8zU17IQTVWYZrzLU_-XR33f1RtJ

Downloading...
From: https://drive.google.com/uc?id=1pmjLR8zU17IQTVWYZrzLU_-XR33f1RtJ
To: /content/identity_CelebA.txt
100% 3.42M/3.42M [00:00<00:00, 203MB/s]


Обученная модель классификации

In [8]:
!gdown 1dN4ozx8EEBiCe3j1tP9bjZHH7HaO4RX9

Downloading...
From (original): https://drive.google.com/uc?id=1dN4ozx8EEBiCe3j1tP9bjZHH7HaO4RX9
From (redirected): https://drive.google.com/uc?id=1dN4ozx8EEBiCe3j1tP9bjZHH7HaO4RX9&confirm=t&uuid=b745d1a6-affd-4bf7-875a-079adb8365eb
To: /content/best_recognition_model_arc.pth
100% 61.6M/61.6M [00:01<00:00, 47.3MB/s]


Обученная модель по определению ключевых точек

In [ ]:
!gdown 1qCzxrZos9ZzmWE_zwZKT5kEnCSUK8n3f

#Формируем данные для подсчета метрик

In [9]:
df = pd.read_csv('/content/identity_CelebA.txt', delim_whitespace=True, header=None,index_col=0)

/tmp/ipython-input-9-1097142516.py:1: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv('/content/identity_CelebA.txt', delim_whitespace=True, header=None,index_col=0)


In [10]:
df.index.name = 'image_name'
df.rename(columns={1:'id'},inplace=True)

Находим файлы, которые не были использованы во время обучения модели

In [11]:
def get_filenames_os(folder_path):
    files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
    return files

In [12]:
all_filenames = get_filenames_os('/content/celeba/img_align_celeba')

In [13]:
used_filenames = get_filenames_os('/content/content/stage_3_dataset')

In [14]:
unused_filenames = list(set(all_filenames) ^ set(used_filenames))

Фильтруем датафрейм  по неиспользованным файлам и id

In [15]:
used_files_dataframe = df.loc[used_filenames]
used_ids = used_files_dataframe['id']
unused_files_dataframe = df.loc[unused_filenames]

unused_filed_unique_id_df = unused_files_dataframe[~unused_files_dataframe["id"].isin(used_ids)]

Формируем `query_dict`, `query_img_names` и `distractors_img_names`

In [16]:
query_ids = unused_filed_unique_id_df.value_counts().sample(30).index.to_list()
query_ids = list(chain(*query_ids))

In [17]:
distractors_img_names = unused_filed_unique_id_df[~unused_filed_unique_id_df['id'].isin(query_ids)].sample(500).index.to_list()

In [18]:
query_dict = {}
query_img_names = []
for id in query_ids:
    img_names = unused_filed_unique_id_df[unused_filed_unique_id_df['id']==id].head(5).index.to_list()
    query_dict[id] = img_names
    query_img_names += img_names

#Обработаем фото перед подачей в модель классификации

##HourglassNet

Написанный за нас ResidualBlock

In [19]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.skip = nn.Identity() if in_channels == out_channels else nn.Conv2d(in_channels, out_channels, 1)

        self.conv1 = nn.Conv2d(in_channels, out_channels // 2, 1)
        self.bn1 = nn.BatchNorm2d(out_channels // 2)
        self.conv2 = nn.Conv2d(out_channels // 2, out_channels // 2, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels // 2)
        self.conv3 = nn.Conv2d(out_channels // 2, out_channels, 1)
        self.bn3 = nn.BatchNorm2d(out_channels)

        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        residual = self.skip(x)
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.bn3(self.conv3(x))
        return self.relu(x + residual)

Рекурсивный HourglassBlock, позволяющий задать его глубину

In [20]:
class HourglassBlock(nn.Module):
    def __init__(self, channels, depth):
        super().__init__()
        self.depth = depth

        self.res = ResidualBlock(channels, channels)

        if depth > 1:
            self.next = HourglassBlock(channels, depth - 1)
        else:
            self.center = ResidualBlock(channels, channels)


        self.downsample = nn.MaxPool2d(kernel_size=2,stride=2)
        self.upsample = nn.Upsample(scale_factor=2, mode='nearest')

    def forward(self, x):
        skip = self.res(x)
        x = self.downsample(skip)

        if self.depth > 1:
            x = self.next(x)
        else:
            x = self.center(x)

        x = self.upsample(x)
        return x + skip

HourglassNet, состоящая из 3 HourglassBlock(каждый глубиной 4), 3 голов и специальных merge слоев(тут мне ChatGPT помог, обосновав их наличие тем, что так лучше обучается модель, чем обычный skip connection).

In [21]:
class HourglassNet(nn.Module):
    def __init__(self, channels=128, depth=4):
        super().__init__()
        self.initial_conv = nn.Conv2d(3, channels, kernel_size=7, stride=1, padding=3)
        self.bn = nn.BatchNorm2d(channels)
        self.relu = nn.ReLU()

        self.hg1 = HourglassBlock(channels,depth)
        self.hg2 = HourglassBlock(channels,depth)
        self.hg3 = HourglassBlock(channels,depth)

        self.merge1 = nn.Conv2d(5, channels, kernel_size=1)
        self.merge2 = nn.Conv2d(5, channels, kernel_size=1)

        self.head1 = nn.Sequential(
            nn.Conv2d(channels, 5, kernel_size=3, padding=1),
            nn.BatchNorm2d(5)
        )
        self.head2 = nn.Sequential(
            nn.Conv2d(channels, 5, kernel_size=3, padding=1),
            nn.BatchNorm2d(5)
        )
        self.head3 = nn.Sequential(
            nn.Conv2d(channels, 5, kernel_size=3, padding=1),
            nn.BatchNorm2d(5)
        )

        self.conv1 = nn.Conv2d(channels, channels, 1)
        self.conv2 = nn.Conv2d(channels, channels, 1)
        self.conv3 = nn.Conv2d(channels, channels, 1)


    def forward(self, x):
        x = self.initial_conv(x)
        x = self.relu(self.bn(x))
        skip1 = self.hg1(x)
        res1 = self.head1(self.conv1(skip1))
        inter1 = skip1 + self.merge1(res1)
        skip2 = self.hg2(inter1)
        res2 = self.head2(self.conv2(skip2))
        inter2 = skip2 + self.merge2(res2)
        x = self.hg3(inter2)
        res3 = self.head3(self.conv3(x))
        return [res1, res2, res3]



In [26]:
model = HourglassNet().to(device)
model.load_state_dict(torch.load('/content/best_model_keypoints.pth', weights_only=True, map_location=device))

<All keys matched successfully>

##Выравнивание лиц

In [27]:
def align_face(image, keypoints, output_size=(112, 112), margin=0.2):
    keypoints = np.array(keypoints, dtype=np.float32).reshape(5, 2)
    center = keypoints.mean(axis=0)

    min_xy = keypoints.min(axis=0)
    max_xy = keypoints.max(axis=0)

    size = (max_xy - min_xy).max() * (1 + margin)
    size = int(size)

    h, w = image.shape[:2]

    left = int(center[0] - size / 2)
    top = int(center[1] - size / 2)
    right = left + size
    bottom = top + size

    # Расчёт нужного паддинга
    pad_left = max(0, -left)
    pad_top = max(0, -top)
    pad_right = max(0, right - w)
    pad_bottom = max(0, bottom - h)

    # Ограничим координаты для кропа
    left = max(0, left)
    top = max(0, top)
    right = min(w, right)
    bottom = min(h, bottom)

    cropped = image[top:bottom, left:right]

    # Добавим padding если нужно
    if any([pad_top, pad_bottom, pad_left, pad_right]):
        cropped = cv2.copyMakeBorder(
            cropped,
            pad_top, pad_bottom, pad_left, pad_right,
            borderType=cv2.BORDER_CONSTANT,
            value=[0, 0, 0]
        )

    # Теперь можно безопасно ресайзить
    resized = cv2.resize(cropped, output_size)
    return resized

##Обработка

In [28]:
class NamedCelebADataset(Dataset):
    def __init__(self, img_dir, img_names, transform=None):
        super().__init__()
        self.img_dir = img_dir
        self.transform = transform
        self.image_names = img_names

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        img_name = self.image_names[idx]
        img_path = os.path.join(self.img_dir, img_name)
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image, img_name

In [29]:
def heatmap_to_coords(hm):
    coords = []
    for i in range(hm.shape[0]):
        y, x = torch.nonzero(hm[i] == hm[i].max(), as_tuple=True)
        if len(x) > 0 and len(y) > 0:
            coords.append((x[0].item(), y[0].item()))
        else:
            coords.append((0, 0))
    return coords

In [32]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])

named_dataset = NamedCelebADataset(
    img_dir='/content/celeba/img_align_celeba',
    transform=transform,
    img_names = query_img_names + distractors_img_names
)

loader = DataLoader(named_dataset)

@torch.no_grad()
def transform_images(loader, model):
    total_res = []

    for i, (image, name) in enumerate(tqdm(loader, leave=False)):
        model.eval()
        image = image.to(device)
        outputs = model(image)
        pred_heatmaps = outputs[-1]
        img = image[0]
        img_vis = TF.to_pil_image(torch.clamp((img + 1) / 2, 0, 1))
        pred = pred_heatmaps[0]
        pred_coords = heatmap_to_coords(pred)
        img_pil = TF.to_pil_image(torch.clamp((img + 1) / 2, 0, 1))
        img_np = np.array(img_pil)
        aligned_image = torch.tensor(align_face(img_np,pred_coords)).permute(2,0,1)
        aligned_image = aligned_image.float() / 255.0

        file_path = os.path.join('/content/cropped_aligned_dataset', name[0])
        vutils.save_image(aligned_image,file_path)



In [34]:
transform_images(loader, model)

  0%|          | 0/645 [00:00<?, ?it/s]

#Модель классификации

In [35]:
backbone = models.resnet18(pretrained=True)
backbone.fc = nn.Identity()
model = nn.Sequential(backbone, nn.Flatten()).to(device)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 169MB/s]


In [36]:
saved_model = torch.load('/content/best_recognition_model_arc.pth', map_location=device)

In [37]:
model.load_state_dict(saved_model['model'])
model = model.to(device)

#Подсчет эмбедингов

In [42]:
img_dir = '/content/cropped_aligned_dataset'

In [39]:
transform = transforms.Compose([
    transforms.Resize((112, 112)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3),
])

In [40]:
@torch.no_grad()
def compute_embeddings(model, images_list):
    '''
    compute embeddings from the trained model for list of images.
    params:
      model: trained nn model that takes images and outputs embeddings
      images_list: list of images paths to compute embeddings for
    output:
      list: list of model embeddings. Each embedding corresponds to images
            names from images_list
    '''
    model.eval()

    embeddings = []

    for img_name in tqdm(images_list):

        img_path = os.path.join(img_dir, img_name)
        image = Image.open(img_path).convert('RGB')

        image_tensor = transform(image)
        image_tensor = image_tensor.unsqueeze(0)
        image_tensor = image_tensor.to(device)

        embedding = model(image_tensor)
        embeddings.append(embedding.squeeze().cpu().detach().numpy())


    return embeddings

In [43]:
query_embeddings = compute_embeddings(model, query_img_names)
distractors_embeddings = compute_embeddings(model, distractors_img_names)

  0%|          | 0/145 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

#Основные функции расчета косинусных расстояний

In [44]:
def compute_cosine_query_pos(query_dict: dict, query_img_names, query_embeddings):
    '''
    compute cosine similarities between positive pairs from query (stage 1)
    params:
      query_dict: dict {class: [image_name_1, image_name_2, ...]}. Key: class in
                  the dataset. Value: images corresponding to that class
      query_img_names: list of images names
      query_embeddings: list of embeddings corresponding to query_img_names
    output:
      list of floats: similarities between embeddings corresponding
                      to the same people from query list
    '''

    embeddings_map = { img_name:embedding for img_name, embedding in zip(query_img_names, query_embeddings) }
    similarities = []

    for image_names in query_dict.values():
        embeddings = [ embeddings_map[img_name] for img_name in image_names ]
        emd_pairs = combinations(embeddings, 2)

        for emb1, emb2 in emd_pairs:
            emb1, emb2 = torch.tensor(emb1), torch.tensor(emb2)
            cos_sim = F.cosine_similarity(emb1.unsqueeze(0), emb2.unsqueeze(0)).item()
            similarities.append(cos_sim)

    return similarities

def compute_cosine_query_neg(query_dict, query_img_names, query_embeddings):
    '''
    compute cosine similarities between negative pairs from query (stage 2)
    params:
      query_dict: dict {class: [image_name_1, image_name_2, ...]}. Key: class in
                  the dataset. Value: images corresponding to that class
      query_img_names: list of images names
      query_embeddings: list of embeddings corresponding to query_img_names
    output:
      list of floats: similarities between embeddings corresponding
                      to different people from query list
    '''
    embeddings_map = { img_name:embedding for img_name, embedding in zip(query_img_names, query_embeddings) }
    similarities = []
    img_id_list = []

    for id, img_names in query_dict.items():
        for img_name in img_names:
            img_id_list.append((img_name,id))

    image_comb = combinations(img_id_list,2)

    for pair1, pair2 in image_comb:
        id1, id2 = pair1[1], pair2[1]
        if id1==id2:
            continue
        img_name1, img_name2 = pair1[0], pair2[0]
        emb1, emb2 = embeddings_map[img_name1], embeddings_map[img_name2]
        emb1, emb2 = torch.tensor(emb1), torch.tensor(emb2)
        cos_sim = F.cosine_similarity(emb1.unsqueeze(0), emb2.unsqueeze(0)).item()
        similarities.append(cos_sim)

    return similarities

def compute_cosine_query_distractors(query_embeddings, distractors_embeddings):
    '''
    compute cosine similarities between negative pairs from query and distractors
    (stage 3)
    params:
      query_embeddings: list of embeddings corresponding to query_img_names
      distractors_embeddings: list of embeddings corresponding to distractors_img_names
    output:
      list of floats: similarities between pairs of people (q, d), where q is
                      embedding corresponding to photo from query, d —
                      embedding corresponding to photo from distractors
    '''
    similarities = []

    for q_emb in query_embeddings:
        q_emb = torch.tensor(q_emb).unsqueeze(0)
        for d_emb in distractors_embeddings:
            d_emb = torch.tensor(d_emb).unsqueeze(0)
            cos_sim = F.cosine_similarity(q_emb, d_emb).item()
            similarities.append(cos_sim)

    return similarities

In [45]:
cosine_query_pos = compute_cosine_query_pos(query_dict, query_img_names,
                                            query_embeddings)
cosine_query_neg = compute_cosine_query_neg(query_dict, query_img_names,
                                            query_embeddings)
cosine_query_distractors = compute_cosine_query_distractors(query_embeddings,
                                                            distractors_embeddings)


Ячейка ниже проверяет, что код работает верно:

In [46]:
test_query_dict = {
    2876: ['1.jpg', '2.jpg', '3.jpg'],
    5674: ['5.jpg'],
    864:  ['9.jpg', '10.jpg'],
}
test_query_img_names = ['1.jpg', '2.jpg', '3.jpg', '5.jpg', '9.jpg', '10.jpg']
test_query_embeddings = [
                    [1.56, 6.45,  -7.68],
                    [-1.1 , 6.11,  -3.0],
                    [-0.06,-0.98,-1.29],
                    [8.56, 1.45,  1.11],
                    [0.7,  1.1,   -7.56],
                    [0.05, 0.9,   -2.56],
]

test_distractors_img_names = ['11.jpg', '12.jpg', '13.jpg', '14.jpg', '15.jpg']

test_distractors_embeddings = [
                    [0.12, -3.23, -5.55],
                    [-1,   -0.01, 1.22],
                    [0.06, -0.23, 1.34],
                    [-6.6, 1.45,  -1.45],
                    [0.89,  1.98, 1.45],
]

test_cosine_query_pos = compute_cosine_query_pos(test_query_dict, test_query_img_names,
                                            test_query_embeddings)
test_cosine_query_neg = compute_cosine_query_neg(test_query_dict, test_query_img_names,
                                            test_query_embeddings)
test_cosine_query_distractors = compute_cosine_query_distractors(test_query_embeddings,
                                                            test_distractors_embeddings)

In [47]:
true_cosine_query_pos = [0.8678237233650096, 0.21226104378511604,
                         -0.18355866977496182, 0.9787437979250561]
assert np.allclose(sorted(test_cosine_query_pos), sorted(true_cosine_query_pos)), \
      "A mistake in compute_cosine_query_pos function"

true_cosine_query_neg = [0.15963231223161822, 0.8507997093616965, 0.9272761484302097,
                         -0.0643994061127092, 0.5412660901220571, 0.701307100338029,
                         -0.2372575528216902, 0.6941032794522218, 0.549425446066643,
                         -0.011982733001947084, -0.0466679194884999]
assert np.allclose(sorted(test_cosine_query_neg), sorted(true_cosine_query_neg)), \
      "A mistake in compute_cosine_query_neg function"

true_cosine_query_distractors = [0.3371426578637511, -0.6866465610863652, -0.8456563512871669,
                                 0.14530087113136106, 0.11410510307646118, -0.07265097629002357,
                                 -0.24097699660707042,-0.5851992679925766, 0.4295494455718534,
                                 0.37604478596058194, 0.9909483738948858, -0.5881093317868022,
                                 -0.6829712976642919, 0.07546364489032083, -0.9130970963915521,
                                 -0.17463101988684684, -0.5229363015558941, 0.1399896725311533,
                                 -0.9258034013399499, 0.5295114163723346, 0.7811585442749943,
                                 -0.8208760031249596, -0.9905139680301821, 0.14969764653247228,
                                 -0.40749654525418444, 0.648660814944824, -0.7432584300096284,
                                 -0.9839696492435877, 0.2498741082804709, -0.2661183373780491]
assert np.allclose(sorted(test_cosine_query_distractors), sorted(true_cosine_query_distractors)), \
      "A mistake in compute_cosine_query_distractors function"

#Расчет метрики

И, наконец, финальная функция, которая считает IR metric:

In [48]:
def compute_ir(cosine_query_pos, cosine_query_neg, cosine_query_distractors,
               fpr=0.1):
    '''
    compute identification rate using precomputer cosine similarities between pairs
    at given fpr
    params:
      cosine_query_pos: cosine similarities between positive pairs from query
      cosine_query_neg: cosine similarities between negative pairs from query
      cosine_query_distractors: cosine similarities between negative pairs
                                from query and distractors
      fpr: false positive rate at which to compute TPR
    output:
      float: threshold for given fpr
      float: TPR at given FPR
    '''

    false_pairs = cosine_query_neg + cosine_query_distractors
    num_false_pairs = len(false_pairs)
    sorted_false_pairs = sorted(false_pairs, reverse=True)
    N = int(fpr * num_false_pairs)

    threshold = sorted_false_pairs[N] if N < len(sorted_false_pairs) else 0
    tp = len(list(filter(lambda sim:sim>threshold,cosine_query_pos)))
    tpr = tp / len(cosine_query_pos)


    return threshold, tpr

И ячейки для ее проверки:

In [49]:
test_thr = []
test_tpr = []
for fpr in [0.5, 0.3, 0.1]:
  x, y = compute_ir(test_cosine_query_pos, test_cosine_query_neg,
                    test_cosine_query_distractors, fpr=fpr)
  test_thr.append(x)
  test_tpr.append(y)

In [50]:
true_thr = [-0.011982733001947084, 0.3371426578637511, 0.701307100338029]
assert np.allclose(np.array(test_thr), np.array(true_thr)), "A mistake in computing threshold"

true_tpr = [0.75, 0.5, 0.5]
assert np.allclose(np.array(test_tpr), np.array(true_tpr)), "A mistake in computing tpr"

#Проверяем на настоящих данных

А в ячейке ниже вы можете посчитать TPR@FPR для датасета с лицами. Давайте, например, посчитаем для значений fpr = [0.5, 0.2, 0.1, 0.05].

In [51]:
thr = []
tpr = []
for fpr in [0.5, 0.2, 0.1, 0.05]:
  x, y = compute_ir(cosine_query_pos, cosine_query_neg,
                    cosine_query_distractors, fpr=fpr)
  thr.append(x)
  tpr.append(y)

In [52]:
thr

[0.9998889565467834,
 0.9999231696128845,
 0.9999340176582336,
 0.9999415278434753]

In [53]:
tpr

[0.5594405594405595,
 0.2692307692307692,
 0.20279720279720279,
 0.13286713286713286]